# DATA CLEANING

## Why is it important?
Poorly cleaned data leads to misleading conclusions and bad models.

## What should it contain?

- Loading data 
- Basic information (.shape, .info(), .describe())
- Handling missing values (e.g., filling in, deleting)
- Searching for and removing duplicates
- Checking types (e.g., dates → datetime, prices → float)
- Filtering out outliers (e.g., $10,000/night on Airbnb is unrealistic)
- Standardization (e.g., currencies, formats)

### Imports

In [ ]:
import os
import sys

import pandas as pd
import numpy as np
from dotenv import load_dotenv

from src.db_connection import DatabaseConfig, DatabaseConnection


dotenv_path = "/Users/erikvida/PycharmProjects/airbnb-price-prediction/.env"
load_dotenv(dotenv_path)

### 1.0 Loading data

In [ ]:
amsterdams_airbnbs_raw_data = pd.read_csv("../data/raw/amsterdam_airbnbs_data.csv")
df= amsterdams_airbnbs_raw_data
df.head()

### 2.0 Basic informations

#### 2.1 Shape

In [ ]:
num_rows, num_columns = df.shape
print(f" Rows: {num_rows}, Columns: {num_columns}")

#### 2.2 Info - Data type of the differnt culomns

In [ ]:
print("\n Data Types:")
df.info()

#### 2.3 Descriptive statistics

In [ ]:
print("\n Descriptive Statistics:")
df.describe().T

### 3.0 Identify and handle missing values

#### 3.1 Identify missing values

In [ ]:
df.replace("?", np.nan, inplace = True)
df.replace("", np.nan, inplace = True)
df.head(5)

#### 3.2 Evaluating for Missing Data

In [ ]:
missing_data = df.isnull()
missing_data

#### 3.3 Count missing values in each column

In [ ]:
for column in missing_data.columns.values.tolist():
    print (missing_data[column].value_counts())
    print("")  

#### 3.4 Remove unneccessary culomns with missing values

In [ ]:
columns_to_drop = [
    "listing_url", "scrape_id", "last_scraped", "source",
    "neighborhood_overview", "picture_url", "host_id", "host_url",
    "host_name", "host_since", "host_location", "host_about", "host_response_time",
    "host_thumbnail_url", "host_picture_url", "host_neighbourhood",
    "host_identity_verified", "neighbourhood_group_cleansed",
    "calendar_updated", "has_availability", "availability_30", "availability_60",
    "availability_90", "availability_365", "calendar_last_scraped",
    "availability_eoy", "estimated_occupancy_l365d", "first_review", "last_review",
    "license", "calculated_host_listings_count",
    "calculated_host_listings_count_entire_homes",
    "calculated_host_listings_count_private_rooms",
    "calculated_host_listings_count_shared_rooms",
    "estimated_revenue_l365d", "reviews_per_month",
    "host_verifications", "latitude", "longitude",
]

df.drop(columns=columns_to_drop, errors="ignore", inplace=True)
df.head()

#### 3.6 Replace missing value freqency

In [ ]:
mode_columns = [
    'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
    'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type'
]

for col in mode_columns:
    if col in df.columns and not df[col].mode().empty:
        df[col] = df[col].fillna(df[col].mode()[0])

df.head(10)

#### 3.7 Replace missing value with mean

In [ ]:
mean_columns = [
    'review_scores_value', 'review_scores_location', 'review_scores_rating',
    'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication'
]

for col in mean_columns:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

df.head(10)


#### 3.8 Replace missing value with random object

In [ ]:
if 'host_response_rate' in df.columns and not df['host_response_rate'].mode().empty:
    df['host_response_rate'] = df['host_response_rate'].fillna(df['host_response_rate'].mode()[0])

df.head(10)

#### 3.9 Drop rows with missing value

In [ ]:
essential_columns = [
    'price', 'neighbourhood', 'neighbourhood_cleansed', 'property_type',
    'bathrooms_text', 'host_has_profile_pic', 'bedrooms', 'beds'
]

for col in essential_columns:
    if col in df.columns:
        df = df.dropna(subset=[col])

df.head(10)

### 4.0 Searching for and removing duplicates

In [ ]:
duplicate_rows = df[df.duplicated(keep=False)]
print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")

### 5.0 Save cleaned data

#### 5.1 Save to csv

In [ ]:
CLEAN_CSV_PATH = "../data/clean/amsterdam_airbnbs_clean_data.csv"
df.to_csv(CLEAN_CSV_PATH, index=False)
print(f"Cleaned data saved to CSV: {CLEAN_CSV_PATH}")

#### 5.2 Save to sql database

In [ ]:
config = DatabaseConfig()
db = DatabaseConnection(config)

TABLE_NAME = "amsterdam_airbnbs_clean_data"

db.write_dataframe(df, TABLE_NAME, if_exists="replace")